In [1]:
with open('y_train.txt') as fopen:
    y = fopen.read().split('\n')
    
with open('x_train.txt') as fopen:
    x = fopen.read().split('\n')
    
with open('y_test.txt') as fopen:
    y.extend(fopen.read().split('\n'))
    
with open('x_test.txt') as fopen:
    x.extend(fopen.read().split('\n'))

In [2]:
import pandas as pd
df = pd.DataFrame({'text':x,'lang':y})
df.head()

,text,lang
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha


In [3]:
selected = df.loc[df.lang.isin(['msa','ind'])]

In [4]:
import re

def augmenter(text,label):
    label = 'zlm' if label == 'msa' else label
    text = text.split()
    half = len(text) // 2
    t = []
    for i in range(len(text) - (half //2)):
        t.append(' '.join(text[i:i + half]))
    return t, [label] * len(t)

In [5]:
X, Y = [], []
for i in range(selected.shape[0]):
    x, y = augmenter(selected.iloc[i,0],selected.iloc[i,1])
    X.extend(x)
    Y.extend(y)

In [6]:
lang = pd.read_csv('sentences.csv',sep='\t')
lang = lang.dropna()
lang.head()

,1,cmn,我們試試看！
0,2,cmn,我该去睡觉了。
1,3,cmn,你在干什麼啊？
2,4,cmn,這是什麼啊？
3,5,cmn,今天是６月１８号，也是Muiriel的生日！
4,6,cmn,生日快乐，Muiriel！


In [7]:
lang.cmn.value_counts().index[:30]

Index(['eng', 'tur', 'rus', 'ita', 'epo', 'deu', 'fra', 'por', 'spa', 'hun',
       'heb', 'jpn', 'ukr', 'ber', 'pol', 'fin', 'mkd', 'nld', 'cmn', 'mar',
       'dan', 'swe', 'srp', 'lat', 'ara', 'ell', 'ces', 'bul', 'lit', 'toki'],
      dtype='object')

In [8]:
selected_langs = ['zlm','eng','ind']
special_langs = ['tur', 'rus', 'ita', 'epo', 'deu', 'fra', 'por', 'spa', 'hun',
       'heb', 'jpn', 'ukr', 'ber', 'pol', 'fin', 'mkd', 'nld', 'cmn', 'mar',
       'dan', 'swe', 'srp', 'lat', 'ara', 'ell', 'ces', 'bul', 'lit', 'toki','kor','fil']

In [9]:
negative_sentences = []
for ln in lang.cmn.unique():
    if ln in selected_langs:
        langs = lang.loc[lang.cmn == ln].iloc[:50000,-1].tolist()
        X.extend(langs)
        Y.extend([ln] * len(langs))
    elif ln in special_langs:
        langs = lang.loc[lang.cmn == ln].iloc[:10000,-1].tolist()
        X.extend(langs)
        Y.extend([ln] * len(langs))
    else:
        print(ln)
        negative_sentences.extend(lang.loc[lang.cmn == ln].iloc[-300:,-1].tolist())

vie
arz
yue
bel
que
swh
nno
wuu
nob
zsm
est
kat
urd
sqi
isl
fry
afr
ron
fao
san
bre
tat
yid
uig
uzb
qya
pes
slk
eus
cycl
acm
tgl
lvs
kaz
hye
hin
ben
cat
bos
hrv
tha
orv
cha
mon
lzh
scn
gle
slv
frm
glg
vol
ain
jbo
ina
nds
mal
tlh
roh
ltz
oss
ido
gla
mlt
sco
ast
jav
oci
ile
xal
tel
sjn
nov
khm
tpi
ang
aze
tgk
tuk
chv
hsb
dsb
bod
sme
cym
mri
ksh
kur
ewe
kab
tpw
udm
lld
pms
lad
grn
mlg
xho
pnb
grc
hat
lao
npi
cor
nah
avk
guj
pan
kir
myv
prg
sux
crs
ckt
bak
hil
cbk
chr
nav
lkt
enm
arq
lin
abk
pcd
rom
gsw
tam
zul
awa
wln
amh
bar
ota
mhr
bho
mrj
osx
pfl
mgm
sna
mah
hau
kan
nog
sin
glv
dng
kal
liv
vro
apc
jdt
fur
che
haw
yor
crh
pdc
ppl
kin
shs
mnw
tet
sah
kum
ngt
nya
pus
hif
mya
moh
wol
tir
lzz
oar
lug
brx
non
mww
hak
nlv
ngu
bua
aym
vec
ibo
tkl
ton
bam
kha
ceb
lou
fuc
smo
gag
lfn
arg
umb
tyv
kjh
oji
cyo
urh
kzj
pam
srd
lmo
swg
mdf
gil
snd
tso
sot
zza
tsn
pau
som
egl
ady
asm
ori
dtp
cho
max
kam
niu
sag
ilo
kaa
fuv
nch
hoc
iba
gbm
sun
war
mvv
pap
ary
kxi
csb
pag
cos
rif
kek
krc
aii
ban
ssw
tvl

In [10]:
X.extend(negative_sentences)
Y.extend(['OTHER'] * len(negative_sentences))

In [11]:
Y[:10]

['ind', 'ind', 'ind', 'ind', 'ind', 'ind', 'ind', 'ind', 'ind', 'ind']

In [12]:
import os
import json
import re
from unidecode import unidecode

def simple_textcleaning_language_detection(string):
    string = re.sub('[^A-Za-z ]+', ' ', unidecode(string))
    string = filter(None, string.split())
    string = [y.strip() for y in string if len(y) > 1]
    return ' '.join(string).lower()

In [13]:
bm = ''
for i in [i for i in os.listdir('crawler') if i.find('isu')>=0]:
    with open('crawler/' + i,'r') as fopen:
        isu = json.load(fopen)
    bm += ' '.join([simple_textcleaning_language_detection(i['summary']) for i in isu if i['language']=='id'])
bm = bm.split()
new_langs = [' '.join(bm[i:i+4]) for i in range(0, len(bm), 4)] 

In [14]:
X.extend(new_langs)
Y.extend(['zlm'] * len(new_langs))

In [15]:
import numpy as np
np.unique(Y, return_counts=True)

(array(['OTHER', 'ara', 'ber', 'bul', 'ces', 'cmn', 'dan', 'deu', 'ell',
        'eng', 'epo', 'fin', 'fra', 'heb', 'hun', 'ind', 'ita', 'jpn',
        'kor', 'lat', 'lit', 'mar', 'mkd', 'nld', 'pol', 'por', 'rus',
        'spa', 'srp', 'swe', 'toki', 'tur', 'ukr', 'zlm'], dtype='<U5'),
 array([ 37910,  10000,  10000,  10000,  10000,  10000,  10000,  10000,
         10000,  50000,  10000,  10000,  10000,  10000,  10000,  55036,
         10000,  10000,   3687,  10000,  10000,  10000,  10000,  10000,
         10000,  10000,  10000,  10000,  10000,  10000,  10000,  10000,
         10000, 128849]))

In [16]:
with open('language-detection-data-v4.json','w') as fopen:
    fopen.write(json.dumps({'text':X,'label':Y}))